In this notebook, we are going to use the setlist.fm API to get the setlists for Dijon and a suite of comparison artists.

In [1]:
# import packages
import pandas as pd
import requests
import os
import json
import time
os.environ["SETLIST_FM_API_KEY"] = os.getenv("SETLIST_API")
API_KEY = os.getenv("SETLIST_API")

/Users/leila/Documents/substack/12-25-dijon-setlists/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# function to get artist setlists
def get_artist_setlist(mbid, start = 0, stop=3):
    # headers for our request
    headers = {"Accept":"application/json",
              "x-api-key":API_KEY}
    # empty list to compile our setlists in
    pages = []
    for i in range(start, stop):
        # page number
        query = {"p":(i)+1}
        try:
            response = requests.get(f"https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists",
                                    headers=headers,
                                    params=query)
            if response.status_code == 200:
                response_items = json.loads(response.content)["setlist"]
                pages.append(response_items)
            elif response.status_code == 404:
                break
            else:
                print(f"Error: {response.status_code}\n\n{response.content}")
        except:
            time.sleep(2)
            response = requests.get(f"https://api.setlist.fm/rest/1.0/artist/{mbid}/setlists",
                                    headers=headers,
                                    params=query)
        
        time.sleep(2)
    return [show for page in pages for show in page]
    

In [3]:
dijon = get_artist_setlist("cceea21d-f3ec-4586-be7a-d2e939c15c78")

In [23]:
mk_gee = get_artist_setlist("262cf0ce-2af2-4686-a8cc-d6b33e97336d",
                            start=0,
                            stop=5)

In [ ]:
## raw data
# all our setlists for dijon
with open("../data/source/dijon/dijon_raw_data.json", 'w') as json_file:
    json.dump(dijon, json_file, indent=4)

# all our setlists for mk.gee
with open("../data/source/mk_gee/mk_gee_raw_data.json", 'w') as json_file:
    json.dump(mk_gee, json_file, indent=4)

## Get all artists

We want to use other artists who played at the Salt Shed since the beginning of 2024 to compare to Dijon. This will be our base comparison dataset and will allow for a wider spread of artists to pull from for visualization purposes.

In [38]:
# let us get some of the artists who toured at salt shed
header = {"Accept":"application/json",
          "x-api-key":API_KEY}
salt_shed_artists_list = []
for i in range(1,25):
    try:
        salt_shed = requests.get("https://api.setlist.fm/rest/1.0/venue/23d07c37/setlists", headers=header, params={"p":i})
        salt_shed_artists_list.append(json.loads(salt_shed.content)["setlist"])
        time.sleep(1)
    except:
        time.sleep(2)
        salt_shed = requests.get("https://api.setlist.fm/rest/1.0/venue/23d07c37/setlists", headers=header, params={"p":i})
        salt_shed_artists_list.append(json.loads(salt_shed.content)["setlist"])

In [ ]:
# collapse list
salt_shed_artists = [subpage for page in salt_shed_artists_list for subpage in page]

In [ ]:
# get into nice dataset for filtering
salt_shed_artist_detail = []

for artist in salt_shed_artists:
    artist_data = {"mbid":artist["artist"]["mbid"],
                                    "name":artist["artist"]["name"],
                                    "event_date":artist["eventDate"]}
    try:
        artist_data.update({"set_length":len(artist["sets"]["set"][0]["song"])})
    except:
        artist_data.update({"set_length":None})
        
    salt_shed_artist_detail.append(artist_data)

In [ ]:
# make into a dataframe and get it to right date format
salt_shed_artist_df = pd.DataFrame(salt_shed_artist_detail)
salt_shed_artist_df["event_date"] = pd.to_datetime(salt_shed_artist_df["event_date"], format="%d-%m-%Y")

In [ ]:
# filter down to tour dates over the last two years where the set wasn't missing
# for a given date, we get the setlist that is the longest... this helps us filter out openers
# we only want the artists latest date at the venue over the last two years
main_acts = salt_shed_artist_df.loc[(salt_shed_artist_df.groupby("event_date")["set_length"].transform("max")==salt_shed_artist_df["set_length"])
                                    &(salt_shed_artist_df["event_date"].dt.year>=2024),
                                    ["mbid", "name", "event_date", "set_length"]].drop_duplicates(["mbid", "name"], keep="first").reset_index(drop=True)

In [ ]:
errors = []
for i in range(len(main_acts)):
    try:
        info = get_artist_setlist(main_acts.loc[i, "mbid"], stop=5)
    except:
        print("Error, will sleep and try requesting data again")
        time.sleep(2)
        try:
            info = get_artist_setlist(main_acts.loc[i, "mbid"], stop=5)
        except:
            print(f"Trouble getting {main_acts.loc[i, "name"]}")
            errors.append(main_acts.loc[i, "mbid"])
            continue
    artist_name = info[0]["artist"]["name"].lower().replace(" ", "_").replace(".", "")
    # all our setlists
    with open(f"../data/source/salt_shed_artists/{artist_name}_raw_data.json", 'w') as json_file:
        json.dump(info, json_file, indent=4)

Some artists who needed more info:

In [4]:
benson = get_artist_setlist("62b914a7-d775-4bb4-bb5e-d46e7df115b5", stop=10)
with open(f"../data/source/salt_shed_artists/benson_boone_raw_data.json", 'w') as json_file:
    json.dump(benson, json_file, indent=4)

In [5]:
jason = get_artist_setlist("857c2983-0a33-466f-8607-28504ba8a6c0", stop=10)
with open(f"../data/source/salt_shed_artists/jason_isbell_and_the_400_unit_raw_data.json", 'w') as json_file:
    json.dump(jason, json_file, indent=4)

In [7]:
marcus = get_artist_setlist("955bc3c8-36d4-47e8-9648-f6728c613583", stop=15)
with open(f"../data/source/salt_shed_artists/marcus_king_raw_data.json", 'w') as json_file:
    json.dump(marcus, json_file, indent=4)

In [9]:
neck_deep = get_artist_setlist("c41dd59f-d805-41df-9e0e-83ec0f9f468e", stop=10)
with open(f"../data/source/salt_shed_artists/neck_deep_raw_data.json", 'w') as json_file:
    json.dump(neck_deep, json_file, indent=4)

In [10]:
font_dc = get_artist_setlist("fd87acc7-e0a0-4a45-bc2a-d2ab5c10be68", stop=10)
with open(f"../data/source/salt_shed_artists/fontaines_dc_raw_data.json", 'w') as json_file:
    json.dump(font_dc, json_file, indent=4)

In [11]:
greensky = get_artist_setlist("199596a3-a1af-49f8-8795-259eff8461fb", stop=10)
with open(f"../data/source/salt_shed_artists/greensky_bluegrass_raw_data.json", 'w') as json_file:
    json.dump(greensky, json_file, indent=4)

In [12]:
extra = get_artist_setlist("d9a9b6aa-c5c5-40dc-95ec-0bff029245c0", stop=10)
with open(f"../data/source/salt_shed_artists/dayseeker_raw_data.json", 'w') as json_file:
    json.dump(extra, json_file, indent=4)

In [13]:
extra = get_artist_setlist("42321e24-42b6-4f08-b0d9-8325ee887a20", stop=10)
with open(f"../data/source/salt_shed_artists/waxahatchee_raw_data.json", 'w') as json_file:
    json.dump(extra, json_file, indent=4)

In [14]:
extra = get_artist_setlist("860d7dc2-618d-437e-a3bb-7b91c2d40095", stop=10)
with open(f"../data/source/salt_shed_artists/ken_carson_raw_data.json", 'w') as json_file:
    json.dump(extra, json_file, indent=4)

In [15]:
extra = get_artist_setlist("aed63829-e917-498b-bb7f-cf76459702dc", stop=10)
with open(f"../data/source/salt_shed_artists/inhaler_raw_data.json", 'w') as json_file:
    json.dump(extra, json_file, indent=4)

In [16]:
extra = get_artist_setlist("606bf117-494f-4864-891f-09d63ff6aa4b", stop=10)
with open(f"../data/source/salt_shed_artists/rise_against_raw_data.json", 'w') as json_file:
    json.dump(extra, json_file, indent=4)